In [2]:
import numpy as np
# import cv2
from PIL import Image
import matplotlib.pyplot as plt
from model import Net

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.models import resnet18
import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
dataset = torchvision.datasets.ImageFolder('data', transform)

In [5]:
n_train = int(len(dataset)*0.6)
n_val = int(len(dataset)*0.2)
n_test = len(dataset) - n_train - n_val

In [6]:
print(n_train)
print(n_val)
print(n_test)
print(n_train + n_val + n_test)

83
27
29
139


In [7]:
torch.manual_seed(0)
batch_size = 10

In [8]:
len(dataset)

139

In [9]:
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [10]:
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size)

In [11]:
pl.seed_everything(0)
net = Net()

Global seed set to 0


In [57]:
trainer = pl.Trainer(max_epochs=10)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [58]:
trainer.fit(net, train_loader, val_loader)


  | Name              | Type     | Params
-----------------------------------------------
0 | feature_extractor | ResNet   | 11.7 M
1 | fc                | Linear   | 2.0 K 
2 | train_acc         | Accuracy | 0     
3 | val_acc           | Accuracy | 0     
4 | test_acc          | Accuracy | 0     
-----------------------------------------------
2.0 K     Trainable params
11.7 M    Non-trainable params
11.7 M    Total params
46.766    Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/yujikawa/.pyenv/versions/3.9.4/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Global seed set to 0


Epoch 0:   0%|          | 0/11 [00:00<?, ?it/s] 

/Users/yujikawa/.pyenv/versions/3.9.4/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Epoch 0:  73%|███████▎  | 8/11 [00:10<00:03,  1.31s/it, loss=1.18, v_num=5] 
Validating: 0it [00:00, ?it/s]
Epoch 1:  73%|███████▎  | 8/11 [00:08<00:03,  1.05s/it, loss=0.841, v_num=5]
Validating: 0it [00:00, ?it/s]
Epoch 2:  73%|███████▎  | 8/11 [00:08<00:03,  1.03s/it, loss=0.584, v_num=5] 
Validating: 0it [00:00, ?it/s]
Epoch 3:  73%|███████▎  | 8/11 [00:08<00:03,  1.01s/it, loss=0.464, v_num=5] 
Validating: 0it [00:00, ?it/s]
Epoch 4:  73%|███████▎  | 8/11 [00:08<00:03,  1.04s/it, loss=0.314, v_num=5] 
Validating: 0it [00:00, ?it/s]
Epoch 5:  73%|███████▎  | 8/11 [00:07<00:02,  1.04it/s, loss=0.135, v_num=5] 
Validating: 0it [00:00, ?it/s]
Epoch 6:  73%|███████▎  | 8/11 [00:07<00:02,  1.04it/s, loss=0.249, v_num=5] 
Validating: 0it [00:00, ?it/s]
Epoch 7:  73%|███████▎  | 8/11 [00:08<00:03,  1.00s/it, loss=0.237, v_num=5] 
Validating: 0it [00:00, ?it/s]
Epoch 8:  73%|███████▎  | 8/11 [00:08<00:03,  1.00s/it, loss=0.107, v_num=5] 
Validating: 0it [00:00, ?it/s]
Epoch 9:  73%|███████

In [60]:
results = trainer.test(test_dataloaders=test_loader)

Testing: 0it [00:00, ?it/s]

/Users/yujikawa/.pyenv/versions/3.9.4/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 100%|██████████| 3/3 [00:03<00:00,  1.21s/it]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.7931034564971924, 'test_loss': 0.29436296224594116}
--------------------------------------------------------------------------------


In [62]:
torch.save(net.state_dict(), 'gen.pt')